# Índice de refração do ar

In [22]:
import numpy as np

# Comprimento de onda do laser (em metros)
lambda_laser = 638.2e-9  # 638.2 nm convertido para metros

# Número de franjas deslocadas
N = 30

# Dados das distâncias medidas (em micrômetros)
distances_micrometers = [10, 9, 8, 9, 9, 10, 10, 11, 9, 9]

# Converter as distâncias para metros
distances_meters = [d * 1e-6 for d in distances_micrometers]

# Calcular a média das distâncias
D_mean = np.mean(distances_meters)

# Calcular o desvio padrão e a incerteza padrão da média
std_dev = np.std(distances_meters, ddof=1)  # Desvio padrão
uncertainty_D = std_dev / np.sqrt(len(distances_meters))  # Incerteza padrão da média

# Calcular o índice de refração do ar
n_air = (N * lambda_laser) / (2 * D_mean)

# Propagação da incerteza para n
uncertainty_n = n_air * (uncertainty_D / D_mean)

# Calcular as incertezas em percentagem
uncertainty_D_percent = (uncertainty_D / D_mean) * 100  # Incerteza de D em %
uncertainty_n_percent = (uncertainty_n / n_air) * 100   # Incerteza de n em %

# Valor de referência para o índice de refração do ar
n_reference = 1.0002718

# Calcular o erro relativo
error_percent = abs((n_air - n_reference) / n_reference) * 100

# Exibir os resultados
print(f"Média das distâncias medidas (D): {D_mean:.3e} m")
print(f"Desvio padrão: {std_dev:.3e} m")
print(f"Incerteza em D: {uncertainty_D:.3e} m ({uncertainty_D_percent:.2f}%)")
print(f"Índice de refração do ar (n): {n_air:.6f}")
print(f"Incerteza em n: {uncertainty_n:.6f} ({uncertainty_n_percent:.2f}%)")
print(f"Erro relativo em relação ao valor de referência (n = {n_reference}): {error_percent:.2f}%")

Média das distâncias medidas (D): 9.400e-06 m
Desvio padrão: 8.433e-07 m
Incerteza em D: 2.667e-07 m (2.84%)
Índice de refração do ar (n): 1.018404
Incerteza em n: 0.028891 (2.84%)
Erro relativo em relação ao valor de referência (n = 1.0002718): 1.81%


# Índice de refração do vidro

In [23]:
import numpy as np

# Dados fornecidos
lambda_laser = 638.2e-9  # Comprimento de onda em metros
h = 5.5e-3  # Espessura do vidro em metros
uncertainty_h = 0.5e-3  # Incerteza em h
n_air = 1.018404  # Índice de refração do ar
uncertainty_n_air = 0.02889090  # Incerteza associada a n_air

# Ângulo do "zero verdadeiro" (em graus)
i_zero = 0.7

# Dados experimentais
angles_degrees = [4.6, 5.7, 6.4, 6.8, 7.0, 7.8]  # Ângulos medidos (em graus)
N_values = [20, 25, 30, 35, 40, 50]  # Número de franjas

# Corrigir os ângulos subtraindo o zero verdadeiro e converter para radianos
angles_corrected_radians = [(i - i_zero) * np.pi / 180 for i in angles_degrees]

# Calcular nv para cada par (N, i)
nv_values = []
uncertainties_nv = []

for i_corr, N in zip(angles_corrected_radians, N_values):
    sin2_i = np.sin(i_corr)**2
    
    # Numerador e denominador
    numerator = h * n_air**2 * sin2_i
    denominator = h * n_air * sin2_i - N * lambda_laser
    
    # Índice de refração do vidro
    nv = numerator / denominator
    nv_values.append(nv)
    
    # Propagação de incertezas
    partial_h = nv * (1 / h)
    partial_n_air = (2 * n_air * sin2_i * h / denominator) - (numerator / denominator**2) * (h * sin2_i)
    partial_i = nv * (2 * np.sin(2 * i_corr) / np.sin(i_corr)**2)
    
    # Incertezas individuais
    uncertainty_n = np.sqrt((partial_h * uncertainty_h)**2 + (partial_n_air * uncertainty_n_air)**2)
    uncertainties_nv.append(uncertainty_n)

# Média ponderada dos valores de nv
weights = [1 / (u**2) for u in uncertainties_nv]  # Pesos inversos das incertezas ao quadrado
nv_mean = np.average(nv_values, weights=weights)
uncertainty_nv_mean = np.sqrt(1 / sum(weights))  # Incerteza da média ponderada

# Valores reais para diferentes tipos de vidro
nv_real_values = {
    "Crown Glass": 1.520,
    "Zinc-Crown Glass": 1.575,
    "Heaviest-Flint Glass": 1.890
}

# Cálculo do erro relativo para cada tipo de vidro
errors = {}
for glass_type, nv_real in nv_real_values.items():
    error_relative = abs(nv_mean - nv_real) / nv_real * 100  # Erro relativo em porcentagem
    errors[glass_type] = error_relative

# Apresentar resultados
print("Resultados para o índice de refração do vidro (nv):")
for i, nv in enumerate(nv_values):
    print(f"N = {N_values[i]:2}, Ângulo corrigido = {angles_corrected_radians[i]:.3f} rad, "
          f"nv = {nv:.3f} ± {uncertainties_nv[i]:.3f} ({(uncertainties_nv[i] / nv) * 100:.2f}%)")

# Apresentar média
print(f"\nValor médio de nv = {nv_mean:.3f} ± {uncertainty_nv_mean:.3f} "
      f"({(uncertainty_nv_mean / nv_mean) * 100:.2f}%)")

# Exibir erro relativo para cada tipo de vidro
print("\nErro relativo para cada tipo de vidro:")
for glass_type, error in errors.items():
    print(f"{glass_type}: {error:.2f}%")

# Determinar o vidro mais provável
min_error_glass = min(errors, key=errors.get)
print(f"\nO vidro mais provável utilizado é: {min_error_glass}")

Resultados para o índice de refração do vidro (nv):
N = 20, Ângulo corrigido = 0.068 rad, nv = 2.007 ± 0.182 (9.09%)
N = 25, Ângulo corrigido = 0.087 rad, nv = 1.629 ± 0.149 (9.16%)
N = 30, Ângulo corrigido = 0.099 rad, nv = 1.558 ± 0.143 (9.19%)
N = 35, Ângulo corrigido = 0.106 rad, nv = 1.574 ± 0.145 (9.18%)
N = 40, Ângulo corrigido = 0.110 rad, nv = 1.639 ± 0.150 (9.16%)
N = 50, Ângulo corrigido = 0.124 rad, nv = 1.624 ± 0.149 (9.16%)

Valor médio de nv = 1.650 ± 0.062 (3.75%)

Erro relativo para cada tipo de vidro:
Crown Glass: 8.56%
Zinc-Crown Glass: 4.77%
Heaviest-Flint Glass: 12.69%

O vidro mais provável utilizado é: Zinc-Crown Glass


# Tamanho da fenda

In [13]:
import numpy as np

# Dados fornecidos
lambda_laser = 638.2e-9  # Comprimento de onda em metros
d = 2.30  # Distância entre a fenda e o plano de observação (em metros)
uncertainty_d = 0.0005  # Incerteza em d (em metros)

# Valores de m, Y e suas incertezas
measurements = [
    {"m": 1, "Y": 0.07 / 2, "uncertainty_Y": 0.0005},  # Y para m=1 (em metros)
    {"m": 2, "Y": 0.146 / 2, "uncertainty_Y": 0.0005},  # Y para m=2 (em metros)
    {"m": 3, "Y": 0.222 / 2, "uncertainty_Y": 0.0005},  # Y para m=3 (em metros)
]

# Calcular b para cada medição
b_values = []
uncertainties_b = []

for data in measurements:
    m = data["m"]
    Y = data["Y"]
    uncertainty_Y = data["uncertainty_Y"]
    
    # Calcular b
    b = abs(m) * d * lambda_laser / Y
    b_values.append(b)
    
    # Propagação de incertezas
    partial_d = abs(m) * lambda_laser / Y
    partial_Y = -abs(m) * d * lambda_laser / (Y**2)
    
    uncertainty_b = np.sqrt((partial_d * uncertainty_d)**2 + (partial_Y * uncertainty_Y)**2)
    uncertainties_b.append(uncertainty_b)

# Média ponderada de b
weights = [1 / (u**2) for u in uncertainties_b]  # Pesos inversos das incertezas ao quadrado
b_mean = np.average(b_values, weights=weights)
uncertainty_b_mean = np.sqrt(1 / sum(weights))  # Incerteza da média ponderada

# Apresentar resultados
print("Resultados para o tamanho da fenda (b):")
for i, b in enumerate(b_values):
    print(f"m = {measurements[i]['m']:2}, Y = {measurements[i]['Y']:.3f} m, "
          f"b = {b:.3e} ± {uncertainties_b[i]:.3e} m ({(uncertainties_b[i] / b) * 100:.2f}%)")

# Apresentar média
print(f"\nValor médio de b = {b_mean:.3e} ± {uncertainty_b_mean:.3e} m "
      f"({(uncertainty_b_mean / b_mean) * 100:.2f}%)")

Resultados para o tamanho da fenda (b):
m =  1, Y = 0.035 m, b = 4.194e-05 ± 5.992e-07 m (1.43%)
m =  2, Y = 0.073 m, b = 4.022e-05 ± 2.756e-07 m (0.69%)
m =  3, Y = 0.111 m, b = 3.967e-05 ± 1.789e-07 m (0.45%)

Valor médio de b = 3.996e-05 ± 1.456e-07 m (0.36%)


# Diâmetro do fio de cabelo

In [15]:
import numpy as np

# Dados fornecidos
lambda_laser = 638.2e-9  # Comprimento de onda em metros
d = 2.30  # Distância entre a fenda e o plano de observação (em metros)
uncertainty_d = 0.0005  # Incerteza em d (em metros)

# Valores de m, Y e suas incertezas
measurements = [
    {"m": 1, "Y": 0.018 / 2, "uncertainty_Y": 0.0005},  # Y para m=1 (em metros)
    {"m": 2, "Y": 0.031 / 2, "uncertainty_Y": 0.0005},  # Y para m=2 (em metros)
    {"m": 3, "Y": 0.046 / 2, "uncertainty_Y": 0.0005},  # Y para m=3 (em metros)
]

# Calcular b para cada medição
b_values = []
uncertainties_b = []

for data in measurements:
    m = data["m"]
    Y = data["Y"]
    uncertainty_Y = data["uncertainty_Y"]
    
    # Calcular b
    b = abs(m) * d * lambda_laser / Y
    b_values.append(b)
    
    # Propagação de incertezas
    partial_d = abs(m) * lambda_laser / Y
    partial_Y = -abs(m) * d * lambda_laser / (Y**2)
    
    uncertainty_b = np.sqrt((partial_d * uncertainty_d)**2 + (partial_Y * uncertainty_Y)**2)
    uncertainties_b.append(uncertainty_b)

# Média ponderada de b
weights = [1 / (u**2) for u in uncertainties_b]  # Pesos inversos das incertezas ao quadrado
b_mean = np.average(b_values, weights=weights)
uncertainty_b_mean = np.sqrt(1 / sum(weights))  # Incerteza da média ponderada

# Apresentar resultados
print("Resultados para o tamanho do cabelo (b):")
for i, b in enumerate(b_values):
    print(f"m = {measurements[i]['m']:2}, Y = {measurements[i]['Y']:.3f} m, "
          f"b = {b:.3e} ± {uncertainties_b[i]:.3e} m ({(uncertainties_b[i] / b) * 100:.2f}%)")

# Apresentar média
print(f"\nValor médio de b = {b_mean:.3e} ± {uncertainty_b_mean:.3e} m "
      f"({(uncertainty_b_mean / b_mean) * 100:.2f}%)")

Resultados para o tamanho do cabelo (b):
m =  1, Y = 0.009 m, b = 1.631e-04 ± 9.061e-06 m (5.56%)
m =  2, Y = 0.015 m, b = 1.894e-04 ± 6.110e-06 m (3.23%)
m =  3, Y = 0.023 m, b = 1.915e-04 ± 4.162e-06 m (2.17%)

Valor médio de b = 1.873e-04 ± 3.216e-06 m (1.72%)


# Diâmetro de um orifício circular

In [17]:
import numpy as np

# Dados fornecidos
lambda_laser = 638.2e-9  # Comprimento de onda do laser (em metros)
n = 1.018404  # Índice de refração do ar (do exercício anterior)
d = 2.30  # Distância entre o orifício e o plano de observação (em metros)
uncertainty_d = 0.0005  # Incerteza em d (em metros)

# Constantes para cada mínimo
constants = {
    1: 1.22,
    2: 2.23,
    3: 3.24,
}

# Valores de q e suas incertezas
measurements = [
    {"m": 1, "q": 0.051 / 2, "uncertainty_q": 0.0005},  # q para o 1º zero (em metros)
    {"m": 2, "q": 0.102 / 2, "uncertainty_q": 0.0005},  # q para o 2º zero (em metros)
    {"m": 3, "q": 0.153 / 2, "uncertainty_q": 0.0005},  # q para o 3º zero (em metros)
]

# Calcular D para cada medição
D_values = []
uncertainties_D = []

for data in measurements:
    m = data["m"]
    q = data["q"]
    uncertainty_q = data["uncertainty_q"]
    constant = constants[m]
    
    # Calcular D
    D = constant * (lambda_laser / n) * (d / q)
    D_values.append(D)
    
    # Propagação de incertezas
    partial_d = constant * (lambda_laser / n) * (1 / q)
    partial_q = -constant * (lambda_laser / n) * (d / q**2)
    
    uncertainty_D = np.sqrt((partial_d * uncertainty_d)**2 + (partial_q * uncertainty_q)**2)
    uncertainties_D.append(uncertainty_D)

# Média ponderada de D
weights = [1 / (u**2) for u in uncertainties_D]  # Pesos inversos das incertezas ao quadrado
D_mean = np.average(D_values, weights=weights)
uncertainty_D_mean = np.sqrt(1 / sum(weights))  # Incerteza da média ponderada

# Apresentar resultados
print("Resultados para o diâmetro da abertura (D):")
for i, D in enumerate(D_values):
    print(f"m = {measurements[i]['m']:2}, q = {measurements[i]['q']:.3f} m, "
          f"D = {D:.3e} ± {uncertainties_D[i]:.3e} m ({(uncertainties_D[i] / D) * 100:.2f}%)")

# Apresentar média
print(f"\nValor médio de D = {D_mean:.3e} ± {uncertainty_D_mean:.3e} m "
      f"({(uncertainty_D_mean / D_mean) * 100:.2f}%)")

Resultados para o diâmetro da abertura (D):
m =  1, q = 0.025 m, D = 6.896e-05 ± 1.352e-06 m (1.96%)
m =  2, q = 0.051 m, D = 6.302e-05 ± 6.180e-07 m (0.98%)
m =  3, q = 0.076 m, D = 6.104e-05 ± 3.992e-07 m (0.65%)

Valor médio de D = 6.205e-05 ± 3.255e-07 m (0.52%)
